In [0]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [1]:
import sys

import warnings
warnings.filterwarnings("ignore")

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, RMSprop, Adam
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

import numpy as np
import pandas as pd
import time

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from keras.layers.pooling import MaxPooling2D

%matplotlib inline
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import math
import io
import zipfile
from PIL import Image

Using TensorFlow backend.


In [0]:
data_dir = "/jet/prs/workspace/mount_point/adiencebenchmark"
meta_path = "/jet/prs/workspace/mount_point/adience_metadata.csv"

In [0]:
os.chdir(data_dir)
file_list = []
for i in os.listdir(data_dir):
    tmp = data_dir + '/' + i
    file_list.append(tmp)
len(file_list)

168

In [0]:
df = pd.read_csv(meta_path)
df = df.replace("Asin", "Asian")
og_im = list(df['original_image'].values)
folder = list(df['folder_name_y'].values)
paths = []
for ind, fold in enumerate(folder):
    paths.append(fold+"/"+og_im[ind])
paths[0]
df['path'] = paths

In [0]:
df_male = df[df['gender_y'] == 'm']
df_female = df[df['gender_y'] == 'f']

In [0]:
print(df_male.shape, df_female.shape)

(8331, 15) (9910, 15)


In [0]:
df_female.head()

,Unnamed: 0,original_image,face_id_y,race,folder_name_y,age_y,gender_y,x,y,dx,dy,tilt_ang,fiducial_yaw_angle,fiducial_score,path
0,0,landmark_aligned_face.1.10257352486_25a8115ef2...,1,White,30601258@N03,"(25, 32)",f,752,1005,542,542,-95,30,86,30601258@N03/landmark_aligned_face.1.102573524...
1,1,landmark_aligned_face.1.10399789333_b862f708bc...,1,White,30601258@N03,"(25, 32)",f,0,366,1185,1428,-90,0,80,30601258@N03/landmark_aligned_face.1.103997893...
2,2,landmark_aligned_face.1.11178623626_e2fdd19e41...,1,White,30601258@N03,"(25, 32)",f,175,435,1052,1051,-75,15,158,30601258@N03/landmark_aligned_face.1.111786236...
3,3,landmark_aligned_face.1.10204731983_643dfbf861...,1,White,30601258@N03,"(25, 32)",f,704,495,720,721,-80,30,145,30601258@N03/landmark_aligned_face.1.102047319...
4,4,landmark_aligned_face.1.10204462444_bee83d77a0...,1,White,30601258@N03,"(25, 32)",f,251,376,1365,1364,-90,15,155,30601258@N03/landmark_aligned_face.1.102044624...


### Splitting data into train, validation and test directories

In [0]:
# # Just taking percentage of paths - both male and female combined
# from sklearn.model_selection import train_test_split

# int_paths, test_paths, int_y, test_y = train_test_split(df['path'], df['gender_y'], test_size=0.30, random_state=42)
# train_paths, val_paths, train_y, val_y = train_test_split(int_paths, int_y, test_size=0.30, random_state=42)
# print(train_paths.shape, val_paths.shape, test_paths.shape)
# print(train_paths.shape, val_y.shape, test_y.shape)

(8952,) (3837,) (5481,)
(8952,) (3837,) (5481,)


In [0]:
# train_paths.values

array(['30601258@N03/landmark_aligned_face.9.9347743973_618c44aebd_o.jpg',
       '115021413@N07/landmark_aligned_face.952.12060014093_e2969294d6_o.jpg',
       '115321157@N03/landmark_aligned_face.1750.12120261706_5c0a4d1c22_o.jpg',
       ...,
       '10001312@N04/landmark_aligned_face.616.8532023384_b7253c1d3c_o.jpg',
       '114841417@N06/landmark_aligned_face.491.12060239583_7a4c83ded1_o.jpg',
       '48135726@N02/landmark_aligned_face.2108.11108639363_b420769f49_o.jpg'],
      dtype=object)

In [0]:
# train_paths.values[0][13:]

'landmark_aligned_face.9.9347743973_618c44aebd_o.jpg'

In [0]:
# train_paths.values[:5]

array(['30601258@N03/landmark_aligned_face.9.9347743973_618c44aebd_o.jpg',
       '115021413@N07/landmark_aligned_face.952.12060014093_e2969294d6_o.jpg',
       '115321157@N03/landmark_aligned_face.1750.12120261706_5c0a4d1c22_o.jpg',
       '8187011@N06/landmark_aligned_face.989.9703457940_e5fd7f644b_o.jpg',
       '8410632@N03/landmark_aligned_face.1783.9189242666_9e71170910_o.jpg'],
      dtype=object)

In [0]:
# train_y.values[:5]

array(['m', 'f', 'm', 'f', 'f'], dtype=object)

In [0]:
# # Create new directory for each
# import shutil, os

# split_path = '/jet/prs/workspace/adience_split/'

# for i, path in enumerate(train_paths.values):
#     if train_y.values[i]=='m': sub = 'm/'
#     else: sub = 'f/'
#     try:
#         path_idx = path.find('/')
#         shutil.copyfile(data_dir+'/'+path, split_path+'train/'+sub+path[path_idx+1:])
#     except:
#         print(path)

114987449@N03/landmark_aligned_face.645.12055036095_9ba0bbcf52_o.jpg


In [0]:
# for i, path in enumerate(val_paths.values):
#     if val_y.values[i]=='m': sub = 'm/'
#     else: sub = 'f/'    
#     try:
#         path_idx = path.find('/')
#         shutil.copyfile(data_dir+'/'+path, split_path+'val/'+sub+path[path_idx+1:])
#     except:
#         print(path)

In [0]:
# for i, path in enumerate(test_paths.values):
#     if test_y.values[i]=='m': sub = 'm/'
#     else: sub = 'f/'    
#     try:
#         path_idx = path.find('/')
#         shutil.copyfile(data_dir+'/'+path, split_path+'test/'+sub+path[path_idx+1:])
#     except:
#         print(path)

### Model

In [0]:
# model from scratch (mfs)
# using the caffe model params as a guide
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.optimizers import SGD, Adam

mfs = Sequential()

# conv1
mfs.add(Conv2D(filters = 96, kernel_size = 7, strides = 4,
              activation = 'relu',input_shape = (224,224,3)))
mfs.add(MaxPooling2D(pool_size = 3, strides = 2))

# conv2
mfs.add(Conv2D(filters = 256, kernel_size = 5, padding = 'same',
              activation = 'relu'))
mfs.add(MaxPooling2D(pool_size = 3, strides = 2))

# conv3
mfs.add(Conv2D(filters = 384, kernel_size = 3,
              activation = 'relu'))
mfs.add(MaxPooling2D(pool_size = 3, strides = 2))

mfs.add(Dense(512, activation='relu'))
mfs.add(Dropout(rate=0.5))
mfs.add(Dense(512, activation='relu'))
mfs.add(Dropout(rate=0.5))
mfs.add(Flatten())
mfs.add(Dense(8))
mfs.add(Dense(2,activation='sigmoid'))

# sgd = SGD(lr=.1, decay=0, momentum=0.9, nesterov=True)
mfs.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
mfs.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 55, 55, 96)        14208     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 384)       885120    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 384)         0         
_________________________________________________________________
dense_1 (Dense)              (None, 5, 5, 512)         197120    
__________

### Training on Entire Dataset

In [0]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
source_path = '/jet/prs/workspace/mount_point/adience_split/'

train_generator = datagen.flow_from_directory(
        source_path+'train',
        target_size=(224, 224),
        batch_size=32)
#         class_mode='binary')

val_generator = datagen.flow_from_directory(
        source_path+'val',
        target_size=(224, 224),
        batch_size=32)
#         class_mode='binary')

test_generator = datagen.flow_from_directory(
        source_path+'test',
        target_size=(224, 224),
        batch_size=32)
#         class_mode='binary')

Found 8951 images belonging to 2 classes.
Found 3837 images belonging to 2 classes.
Found 5481 images belonging to 2 classes.


In [0]:
# Changing target to categorical - isthis required here?
# train_y = to_categorical(train_y.map({'f':0,'m':1}), num_classes=2)
# val_y = to_categorical(val_y.map({'f':0,'m':1}), num_classes=2)
# test_y = to_categorical(test_y.map({'f':0,'m':1}), num_classes=2)
# print(train_y.shape, val_y.shape, test_y.shape)

In [0]:
history = mfs.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=50,
        validation_data=val_generator,
        validation_steps=80)

Epoch 1/50
100/100 [==============================] - 145s 1s/step - loss: 0.6900 - acc: 0.5440 - val_loss: 0.6917 - val_acc: 0.5299
Epoch 2/50
100/100 [==============================] - 133s 1s/step - loss: 0.6882 - acc: 0.5402 - val_loss: 0.6835 - val_acc: 0.5744
Epoch 3/50
100/100 [==============================] - 137s 1s/step - loss: 0.6751 - acc: 0.5723 - val_loss: 0.6718 - val_acc: 0.5537
Epoch 4/50
100/100 [==============================] - 131s 1s/step - loss: 0.6539 - acc: 0.6059 - val_loss: 0.6588 - val_acc: 0.5971
Epoch 5/50
100/100 [==============================] - 133s 1s/step - loss: 0.6537 - acc: 0.5944 - val_loss: 0.6346 - val_acc: 0.6340
Epoch 6/50
100/100 [==============================] - 134s 1s/step - loss: 0.6195 - acc: 0.6525 - val_loss: 0.6016 - val_acc: 0.6725
Epoch 7/50
100/100 [==============================] - 132s 1s/step - loss: 0.6058 - acc: 0.6654 - val_loss: 0.6279 - val_acc: 0.6568
Epoch 8/50
100/100 [==============================] - 131s 1s/step - 

In [0]:
mfs.save('mfs_train_full.h5')

In [0]:
pwd

'/jet/prs/workspace/mount_point/adiencebenchmark'

In [0]:
cd ../../

/jet/prs/workspace


In [0]:
pwd

'/jet/prs/workspace'

In [0]:
import pickle

with open('trainHistoryDict', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)